In [1]:
# Traemos la libreria VISA
import pyvisa as visa
# Traemos matplotlib para poder graficar
import matplotlib.pyplot as plt
# Agreamos el path de las librerias
import sys
sys.path.insert(0, '../instrumentacion_virtual/Libreria')
import platform
# Traemos el generador
from generadores_arbitrarios import Agilent33512A
# Siempre util numpy y scipy...
import numpy as np
from scipy import signal

import pandas as pd
import time

import serial


# Definiciones y Path

In [2]:
# Device a utilizar de la lista
USE_DEVICE = 1

# Puerto del Arduino
PUERTO_ARDUINO = '/dev/ttyUSB1'

# Limites de tensiones 
V_INI = 0.0
V_FIN = 3.3
# Pasos a medir
N_STEP = 32
# MEdiciones por paso (para tio A)
MED_POR_PASO = 10



### Lectura del paquete

In [3]:
def read_pckg(serial_obj):
    
    serial_obj.write(b'a')
    
    data = serial_obj.read(1)
    while(data != b'<'):
        data = serial_obj.read(1)
            
    # Leo datos
    datos_in = list()
    while(True):
        data = serial_obj.read(1)
        if data != b'>':
            datos_in.append(data)
        else:
            break
        
    # Paso a un string
    string_dato = b''.join(datos_in).decode("utf-8")
    # Lo separo
    dato_splitted = string_dato.split(';')
    # Lo paso a int
    dato_a = int(dato_splitted[0])

    return dato_a
    

# Inicializacion

### Inicializo Instrumento

In [4]:

# Abrimos el instrumento con el backend correcto
platforma = platform.platform()
print(platforma)
if 'pyvisa' in sys.modules:
	rm=visa.ResourceManager('@py')
elif 'visa' in sys.modules:
	rm=visa.ResourceManager('@ni')
else:
	error()

# Instancio el instrumento
instrument_handler=rm.open_resource(rm.list_resources()[USE_DEVICE])
MiGenArb = Agilent33512A(instrument_handler)

# Informamos el modelo del generador conectado
print("Esta conectado un %s"%MiGenArb.INSTR_ID)

Linux-4.15.0-55-generic-x86_64-with-Ubuntu-18.04-bionic
Esta conectado un Agilent Technologies,33522A,MY50002138,1.12-1.19-1.01-50-00



### Inicializo Puerto serie (Arduino)

In [5]:
ser = serial.Serial(PUERTO_ARDUINO, timeout=1, baudrate=9600)
# espero
time.sleep(1)

# Medición

In [6]:
# Campos a medir
campos = ['V_Med_mean', 'V_Med_std']
# Lista de valores de tensión
val_tension = np.linspace(V_INI,V_FIN,N_STEP)

In [ ]:
# Reservo memoria
datos_medidos = np.zeros((len(val_tension), len(campos)))

# Mido
for ids_v, v_act in enumerate(val_tension):
    
    # Seteo generador
    MiGenArb.continua(v_act)
    # espero
    time.sleep(0.5)
    
    # Mido N veces el punto
    dato_aux = np.zeros(MED_POR_PASO)
    for i in range(MED_POR_PASO):
        # Leemos
        dato_1 = read_pckg(ser)
        dato_aux[i] = float(dato_1)
        
        # esperamos
        time.sleep(0.5)
        
    # Saco la media y el desvio y los guardo
    datos_medidos[ids_v,0] = np.mean(dato_aux)
    datos_medidos[ids_v,1] = np.std(dato_aux)
    
    print("V Gen = %0.5f, V Med mean = %0.5f, V Med avg = %0.5f"%(v_act,
                                                                  datos_medidos[ids_v,0], 
                                                                  datos_medidos[ids_v,1]))
        
    

V Gen = 0.00000, V Med mean = 0.00000, V Med avg = 0.00000
V Gen = 0.10645, V Med mean = 21.20000, V Med avg = 0.40000
V Gen = 0.21290, V Med mean = 44.90000, V Med avg = 0.30000
V Gen = 0.31935, V Med mean = 68.60000, V Med avg = 0.48990
V Gen = 0.42581, V Med mean = 92.00000, V Med avg = 0.00000
V Gen = 0.53226, V Med mean = 115.80000, V Med avg = 0.40000
V Gen = 0.63871, V Med mean = 139.00000, V Med avg = 0.00000
V Gen = 0.74516, V Med mean = 162.50000, V Med avg = 0.50000
V Gen = 0.85161, V Med mean = 186.00000, V Med avg = 0.00000
V Gen = 0.95806, V Med mean = 209.60000, V Med avg = 0.48990
V Gen = 1.06452, V Med mean = 233.40000, V Med avg = 0.48990
V Gen = 1.17097, V Med mean = 256.80000, V Med avg = 0.40000
V Gen = 1.27742, V Med mean = 280.20000, V Med avg = 0.40000
V Gen = 1.38387, V Med mean = 303.60000, V Med avg = 0.48990
V Gen = 1.49032, V Med mean = 327.00000, V Med avg = 0.00000
V Gen = 1.59677, V Med mean = 351.00000, V Med avg = 0.00000
V Gen = 1.70323, V Med mean = 

### Armo un Dataframe

In [ ]:
df = pd.DataFrame(datos_medidos, index=val_tension, columns =campos)

### Ploteo

In [ ]:
df.plot(y='V_Med_mean', use_index=True)
plt.show()

# Cierro Todo

In [ ]:
MiGenArb.close()

In [ ]:
ser.close()

# Codigo Aduino